<a href="https://colab.research.google.com/github/mkhaas/CCPCiscolive/blob/master/hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install h5py pyyaml

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

!pip install tensorflow==2.0.0-alpha0
import tensorflow as tf
import numpy as np
from tensorflow import keras

tf.__version__

In [0]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])

In [0]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [0]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [0]:
model.fit(xs, ys, epochs=500)

In [0]:
print(model.predict([10.0]))

In [0]:
print('\ntrain_labels.shape: {}, of {}'.format(xs, xs.dtype))
print('test_labels.shape: {}, of {}'.format(ys.shape, ys.dtype))
print(model.inputs)
print(model.outputs)
inputs={'xs': model.input}
print(inputs)
outputs={t.name:t for t in model.outputs}
print(outputs)


In [0]:
#https://www.tensorflow.org/alpha/tutorials/keras/save_and_restore_models
#https://www.tensorflow.org/tfx/serving/signature_defs
#https://www.tensorflow.org/api_docs/python/tf/saved_model/simple_save
#https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple
#https://colab.research.google.com/github/tensorflow/serving/blob/master/tensorflow_serving/g3doc/tutorials/Serving_REST_simple.ipynb#scrollTo=0w5Rq8SsgWE6
#https://www.tensorflow.org/guide/saved_model
#https://www.tensorflow.org/tfx/serving/setup
#https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/signature_constants.py
#https://www.tensorflow.org/tfx/serving/serving_basic
#https://www.tensorflow.org/tfx/serving/serving_advanced
#https://www.google.com/search?q=examples+of+signature_def_map&oq=examples+of+signature_def_map&aqs=chrome..69i57.5039j0j9&sourceid=chrome&ie=UTF-8
#https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/meta_graph.proto
#https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_saved_model.py
#https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_saved_model.py
#https://www.tensorflow.org/tfx/serving/serving_basic 
#Multiple GPUS
#https://www.tensorflow.org/guide/keras


import tempfile
import sys

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}
  

#tensor_info_x = tf.saved_model.utils.build_tensor_info(xs)
#tensor_info_y = tf.saved_model.utils.build_tensor_info(ys)
  
## Prediction Signature  
#prediction_signature = (
#      tf.saved_model.signature_def_utils.build_signature_def(
          #inputs={'input': tensor_info_x},
          #outputs={'output': tensor_info_y},
 #         inputs={'xs': model.inputs},
 #         outputs={'ys': model.outputs},
 #         method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

#sess = tf.InteractiveSession()
#https://www.tensorflow.org/tfx/serving/serving_basic
#builder = tf.saved_model.builder.SavedModelBuilder(export_path)
#builder.add_meta_graph_and_variables(
#    sess, [tf.saved_model.tag_constants.SERVING],
#    signature_def_map={
#        'predict_images':
#            prediction_signature,
#    })

#builder.save()
#print('Done exporting!')


#tf.saved_model.simple_save(
#    keras.backend.get_session(),
#    export_path,
#    inputs={'xs': model.input},
#    outputs={t.name:t for t in model.outputs})

tf.keras.experimental.export_saved_model(model, export_path)
export_path
print('\nSaved model:')
!ls -l {export_path}

In [0]:
!ls /tmp



In [0]:
!ls /tmp/1

In [0]:
new_model = tf.keras.experimental.load_from_saved_model(export_path)
new_model.summary()

In [0]:
print(model.predict([10.0]))

In [0]:
!ls -ltr /tmp/1

In [0]:
!ls -ltr /tmp/1/variables

In [0]:
!ls -ltr /tmp/1/assets

In [0]:
#from google.colab import files
#files.download("./saved_models/1556802420/saved_model.pb")

In [0]:
!saved_model_cli show --dir {export_path} --all

In [0]:
#https://colab.research.google.com/github/tensorflow/serving/blob/master/tensorflow_serving/g3doc/tutorials/Serving_REST_simple.ipynb#scrollTo=LU4GDF_aYtfQ


# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [0]:
!apt-get install tensorflow-model-server

In [0]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=sm_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [0]:
!tail server.log

In [0]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": [10.0]})
data

In [0]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/sm_model:predict', data=data, headers=headers)
json_response
#predictions = json.loads(json_response.text)['predictions']
#predictions

In [0]:
#https://stackoverflow.com/questions/52671138/tensorflow-serving-rest-api-returns-malformed-request-error
!curl http://localhost:8501/v1/models/sm_model

In [0]:
!tensorflow_model_server --version

In [0]:
!tail server.log

In [0]:
!curl -X POST http://localhost:8501/v1/models/sm_model/versions/1:predict -d '{"instances": [[10.0]]}'

In [0]:
#https://www.tensorflow.org/tfx/tutorials/serving/rest_simple
#https://www.tensorflow.org/tfx/serving/api_rest
!curl -d '"instances": [10.0]' -X POST http://localhost:8501/v1/models/sm_model:predict

In [0]:
 #https://www.tensorflow.org/tfx/serving/setup
 #https://www.tensorflow.org/tfx/serving/docker
#https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/docker.md 
#https://github.com/CiscoAI/KFLab/tree/master/pipelines/tf-mnist
#http://blog.oddbit.com/post/2014-08-11-four-ways-to-connect-a-docker/

In [0]:
!sudo apt-get install -y apt-transport-https ca-certificates unzip
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

!sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/ubuntu \
   $(lsb_release -cs) \
   stable"
!sudo apt-get update


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20180409).
unzip is already the newest version (6.0-21ubuntu1).
apt-transport-https is already the newest version (1.6.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
OK
Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x8

In [0]:
!sudo apt-get -y install docker-ce

!sudo service docker restart

!sudo docker run hello-world

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  apparmor aufs-tools cgroupfs-mount containerd.io docker-ce-cli iptables
  libip6tc0 libiptc0 libmnl0 libnetfilter-conntrack3 libnfnetlink0
  libxtables12 pigz
Suggested packages:
  apparmor-profiles-extra apparmor-utils
The following NEW packages will be installed:
  apparmor aufs-tools cgroupfs-mount containerd.io docker-ce docker-ce-cli
  iptables libip6tc0 libiptc0 libmnl0 libnetfilter-conntrack3 libnfnetlink0
  libxtables12 pigz
0 upgraded, 14 newly installed, 0 to remove and 16 not upgraded.
Need to get 51.6 MB of archives.
After this operation, 247 MB of additional disk space will be used.
Get:1 https://download.docker.com/linux/ubuntu bionic/stable amd64 containerd.io amd64 1.2.5-1 [19.9 MB]
G

In [0]:
!which docker


/usr/bin/docker


In [0]:
!sudo service docker restart

In [0]:
!docker --version


Docker version 18.09.6, build 481bc77


In [0]:
!docker pull tensorflow/serving

Using default tag: latest
Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [0]:
!service docker status

 * Docker is not running


In [0]:
!service chkconfig --list

chkconfig: unrecognized service


In [0]:
#http://blog.oddbit.com/post/2014-08-11-four-ways-to-connect-a-docker/
#https://www.kubeflow.org/docs/pipelines/sdk/component-development/
#https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/docker.md
#https://github.com/StefanoFioravanzo/kale
#https://github.com/CiscoAI/KFLab/tree/master/pipelines/tf-mnist
#https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_saved_model.py
#https://www.tensorflow.org/tfx/serving/serving_basic
#https://www.tensorflow.org/tfx/serving/signature_defs
#https://www.tensorflow.org/alpha/tutorials/keras/save_and_restore_models
#https://www.tensorflow.org/tfx/serving/tutorials/Serving_REST_simple